In [1]:
"""
FEniCS tutorial demo program: Incompressible Navier-Stokes equations
for flow around a cylinder using the Incremental Pressure Correction
Scheme (IPCS).

  u' + u . nabla(u)) - div(sigma(u, p)) = f
                                 div(u) = 0
"""

from __future__ import print_function
from fenics import *
from mshr import *
import numpy as np

T = 5.0            # final time
num_steps = 5000   # number of time steps
dt = T / num_steps # time step size
mu = 0.001         # dynamic viscosity
rho = 1            # density

# Create mesh
channel = Rectangle(Point(0, 0), Point(2.2, 0.41))
cylinder = Circle(Point(0.2, 0.2), 0.05)
domain = channel - cylinder
mesh = generate_mesh(domain, 64)

# Define function spaces
V = VectorFunctionSpace(mesh, 'P', 2)
Q = FunctionSpace(mesh, 'P', 1)

# Define boundaries
inflow   = 'near(x[0], 0)'
outflow  = 'near(x[0], 2.2)'
walls    = 'near(x[1], 0) || near(x[1], 0.41)'
cylinder = 'on_boundary && x[0]>0.1 && x[0]<0.3 && x[1]>0.1 && x[1]<0.3'

# Define inflow profile
inflow_profile = ('4.0*1.5*x[1]*(0.41 - x[1]) / pow(0.41, 2)', '0')

# Define boundary conditions
bcu_inflow = DirichletBC(V, Expression(inflow_profile, degree=2), inflow)
bcu_walls = DirichletBC(V, Constant((0, 0)), walls)
bcu_cylinder = DirichletBC(V, Constant((0, 0)), cylinder)
bcp_outflow = DirichletBC(Q, Constant(0), outflow)
bcu = [bcu_inflow, bcu_walls, bcu_cylinder]
bcp = [bcp_outflow]

In [2]:
# Define trial and test functions
u = TrialFunction(V)
v = TestFunction(V)
p = TrialFunction(Q)
q = TestFunction(Q)

# Define functions for solutions at previous and current time steps
u_n = Function(V)
u_  = Function(V)
p_n = Function(Q)
p_  = Function(Q)

# Define expressions used in variational forms
U  = 0.5*(u_n + u)
n  = FacetNormal(mesh)
f  = Constant((0, 0))
k  = Constant(dt)
mu = Constant(mu)
rho = Constant(rho)

# Define symmetric gradient
def epsilon(u):
    return sym(nabla_grad(u))

# Define stress tensor
def sigma(u, p):
    return 2*mu*epsilon(u) - p*Identity(len(u))

# Define variational problem for step 1
F1 = rho*dot((u - u_n) / k, v)*dx \
   + rho*dot(dot(u_n, nabla_grad(u_n)), v)*dx \
   + inner(sigma(U, p_n), epsilon(v))*dx \
   + dot(p_n*n, v)*ds - dot(mu*nabla_grad(U)*n, v)*ds \
   - dot(f, v)*dx
a1 = lhs(F1)
L1 = rhs(F1)

# Define variational problem for step 2
a2 = dot(nabla_grad(p), nabla_grad(q))*dx
L2 = dot(nabla_grad(p_n), nabla_grad(q))*dx - (1/k)*div(u_)*q*dx

# Define variational problem for step 3
a3 = dot(u, v)*dx
L3 = dot(u_, v)*dx - k*dot(nabla_grad(p_ - p_n), v)*dx

# Assemble matrices
A1 = assemble(a1)
A2 = assemble(a2)
A3 = assemble(a3)

# Apply boundary conditions to matrices
[bc.apply(A1) for bc in bcu]
[bc.apply(A2) for bc in bcp]

[None]

In [3]:
outdir = 'solutions/navier_cylinder'

# Create XDMF files for visualization output
xdmffile_u = XDMFFile(outdir + '/velocity.xdmf')
xdmffile_p = XDMFFile(outdir + '/pressure.xdmf')

# Create time series (for use in reaction_system.py)
timeseries_u = TimeSeries(outdir + '/velocity_series')
timeseries_p = TimeSeries(outdir + '/pressure_series')

# Save mesh to file (for use in reaction_system.py)
File(outdir + '/cylinder.xml.gz') << mesh

# Create progress bar
progress = Progress('Time-stepping')
set_log_level(LogLevel.PROGRESS)

In [ ]:
# Time-stepping
t = 0
for n in range(num_steps):

    # Update current time
    t += dt

    # Step 1: Tentative velocity step
    b1 = assemble(L1)
    [bc.apply(b1) for bc in bcu]
    solve(A1, u_.vector(), b1, 'bicgstab', 'hypre_amg')

    # Step 2: Pressure correction step
    b2 = assemble(L2)
    [bc.apply(b2) for bc in bcp]
    solve(A2, p_.vector(), b2, 'bicgstab', 'hypre_amg')

    # Step 3: Velocity correction step
    b3 = assemble(L3)
    solve(A3, u_.vector(), b3, 'cg', 'sor')

    # Plot solution
    plot(u_, title='Velocity')
    plot(p_, title='Pressure')

    # Save solution to file (XDMF/HDF5)
    xdmffile_u.write(u_, t)
    xdmffile_p.write(p_, t)

    # Save nodal values to file
    timeseries_u.store(u_.vector(), t)
    timeseries_p.store(p_.vector(), t)

    # Update previous solution
    u_n.assign(u_)
    p_n.assign(p_)

    # Update progress bar
#     progress.update(t / T)
    print('u max:', u_.vector().get_local().max())

u max: 2.739542375402943
u max: 3.0540548165201726
u max: 2.545058468418586
u max: 2.3464290141232746
u max: 2.300781804078773
u max: 2.2423722757128917
u max: 2.2112930630803636
u max: 2.2041650359059055
u max: 2.1777773511515877
u max: 2.138551389930672
u max: 2.0924466533976704
u max: 2.0444494860362257
u max: 1.9984657727110677
u max: 1.9572729987292519
u max: 1.9729039513861233
u max: 1.987092097913795
u max: 1.9968313882642998
u max: 2.002679514796774
u max: 2.0052827734914587
u max: 2.005331359360057
u max: 2.003512343559777
u max: 2.0004609703719316
u max: 1.9967185825725537
u max: 1.9927052828586302
u max: 1.988710746932812
u max: 1.9849017476738906
u max: 1.9813420310324186
u max: 1.9780192500115419
u max: 1.97487397315767
u max: 1.9718266899297663
u max: 1.9687998958814503
u max: 1.9657335941185767
u max: 1.9625937674139737
u max: 1.9593744174308623
u max: 1.9560945119698216
u max: 1.9527915665121187
u max: 1.949513627027806
u max: 1.946311192919136
u max: 1.9432302285494334

u max: 2.179772532969079
u max: 2.1797701490106296
u max: 2.1797680045370877
u max: 2.1797659150291673
u max: 2.1797638090829357
u max: 2.1797617306565806
u max: 2.179759823822554
u max: 2.1797583033653485
u max: 2.1797574164958626
u max: 2.1797574019831774
u max: 2.1797584530296183
u max: 2.179760689287938
u max: 2.179764141733371
u max: 2.1797687519527633
u max: 2.179774385146377
u max: 2.17978085410072
u max: 2.1797879498757378
u max: 2.179795474155552
u max: 2.179803268219697
u max: 2.1798112342591964
u max: 2.1798193461355853
u max: 2.179827648425985
u max: 2.1798362444345516
u max: 2.1798452755014552
u max: 2.1798548951660672
u max: 2.17986524237752
u max: 2.1798764179225536
u max: 2.179888467588841
u max: 2.179901374423623
u max: 2.179915060975258
u max: 2.1799294008508854
u max: 2.179944237521957
u max: 2.179959407266241
u max: 2.1799747625945396
u max: 2.1799901925392087
u max: 2.180005636758844
u max: 2.180021091437723
u max: 2.1800366062639016
u max: 2.1800522731528185
u max

u max: 2.1391901528034105
u max: 2.1390352337243446
u max: 2.138884135374705
u max: 2.1387368992895066
u max: 2.1385935658799813
u max: 2.1384541745049424
u max: 2.1383187635760463
u max: 2.1381873706761154
u max: 2.138060032667765
u max: 2.137936785771762
u max: 2.1378176655999903
u max: 2.1377027071359627
u max: 2.137591944664883
u max: 2.137485411663781
u max: 2.1373831406688657
u max: 2.1372851631406435
u max: 2.1371915093474168
u max: 2.1371022082839897
u max: 2.1370172876361617
u max: 2.1369367737934826
u max: 2.1368606919045443
u max: 2.1367890659620445
u max: 2.136721918900285
u max: 2.1366592726862215
u max: 2.1366011483869967
u max: 2.1365475662014823
u max: 2.136498545450058
u max: 2.1364541045241685
u max: 2.136414260804067
u max: 2.136379030558351
u max: 2.1363484288414383
u max: 2.136322469404978
u max: 2.1363011646361136
u max: 2.1362845255305416
u max: 2.1362725617019853
u max: 2.1362652814235634
u max: 2.1362626916913037
u max: 2.1362647982967955
u max: 2.1362716058951

u max: 2.1457157109662752
u max: 2.1453217049423694
u max: 2.1449285103251428
u max: 2.1445362552159106
u max: 2.1441450680367695
u max: 2.143755077491489
u max: 2.1433664125261203
u max: 2.142979202289344
u max: 2.142593576092527
u max: 2.1422096633694294
u max: 2.141827593635578
u max: 2.1414474964472454
u max: 2.1410695013600187
u max: 2.1406937378870374
u max: 2.1403203354568454
u max: 2.139949423370977
u max: 2.139581130761299
u max: 2.13921558654718
u max: 2.1388529193925043
u max: 2.1384932576626037
u max: 2.138136729381079
u max: 2.137783462186516
u max: 2.1374335832891007
u max: 2.1370872194270842
u max: 2.136744496823086
u max: 2.13640554114024
u max: 2.13607047743817
u max: 2.1357394301288206
u max: 2.135412522932184
u max: 2.1350898788319372
u max: 2.1347716200310605
u max: 2.1344578679074355
u max: 2.1341487429695123
u max: 2.1338443648119707
u max: 2.1335448520714575
u max: 2.1332503223823442
u max: 2.132960892332498
u max: 2.1326766774190626
u max: 2.1323977920042334
u m

u max: 2.1794767681272105
u max: 2.1788888611303245
u max: 2.1782946843387787
u max: 2.1776944357434607
u max: 2.177088316563612
u max: 2.1764765311951098
u max: 2.1758592871590543
u max: 2.1752367950509814
u max: 2.1746092684908485
u max: 2.17397692407374
u max: 2.1733399813210452
u max: 2.1726986626317246
u max: 2.1720531932331983
u max: 2.1714038011313055
u max: 2.17075071705881
u max: 2.170094174421949
u max: 2.1694344092445963
u max: 2.16877166010973
u max: 2.1681061680979754
u max: 2.167438176723105
u max: 2.166767931864479
u max: 2.166095681696505
u max: 2.165421676615264
u max: 2.1647461691624756
u max: 2.164069413947057
u max: 2.1633916675645315
u max: 2.162713188514551
u max: 2.1620342371167607
u max: 2.1613550754253485
u max: 2.1606759671423625
u max: 2.1599971775300277
u max: 2.159318973322133
u max: 2.1586416226344762
u max: 2.1579653948742927
u max: 2.157290560648478
u max: 2.1566173916703097
u max: 2.155946160664352
u max: 2.1552771412691034
u max: 2.154610607936963
u ma

...

u max: 2.320021235707366
u max: 2.3193978779661415
u max: 2.3187407190501337
u max: 2.318049913307669
u max: 2.317325627324633
u max: 2.316568039991894
u max: 2.315777342555495
u max: 2.3149537386495638
u max: 2.3140974443135103
u max: 2.3132086879965574
u max: 2.312287710554132
u max: 2.311334765241503
u max: 2.310350117710456
u max: 2.3093340460142664
u max: 2.308286840624795
u max: 2.3072088044635053
u max: 2.306100252945477
u max: 2.304961514032743
u max: 2.30379292829082
u max: 2.30259484894062
u max: 2.301367641897413
u max: 2.3001116857892994
u max: 2.2988273719496193
u max: 2.2975151043807185
u max: 2.296175299689755
u max: 2.2948083870002525
u max: 2.293414807845205
u max: 2.291995016048374
u max: 2.2905494775997597
u max: 2.289078670529322
u max: 2.2875830847804983
u max: 2.2860632220823742
u max: 2.2845195958173896
u max: 2.2829527308806674
u max: 2.2813631635274216
u max: 2.279751441206567
u max: 2.2781181223807967
u max: 2.2764637763357403
u max: 2.274788982982552
u max: 2

u max: 2.325478684269633
u max: 2.325113955684898
u max: 2.324714284120092
u max: 2.3242797346381976
u max: 2.323810383813463
u max: 2.3233063198755843
u max: 2.322767642851943
u max: 2.3221944647050754
u max: 2.3215869094623365
u max: 2.3209451133345937
u max: 2.32026922482068
u max: 2.3195594047946466
u max: 2.3188158265735925
u max: 2.318038675964857
u max: 2.3172281512930066
u max: 2.3163844634085637
u max: 2.3155078356823475
u max: 2.3145985039906685
u max: 2.313656716697704
u max: 2.312682734641757
u max: 2.311676831131458
u max: 2.3106392919565746
u max: 2.309570415415795
u max: 2.308470512360972
u max: 2.3073399062543243
u max: 2.306178933232373
u max: 2.304987942168462
u max: 2.303767294724955
u max: 2.3025173653868576
u max: 2.3012385414704553
u max: 2.2999312231034823
u max: 2.2985958231766213
u max: 2.2972327672692567
u max: 2.295842493554683
u max: 2.2944254526909855
u max: 2.2929821077034416
u max: 2.2915129338626943
u max: 2.290018418560557
u max: 2.288499061182828
u max

u max: 2.327989877317274
u max: 2.3278951979130715
u max: 2.327765275915893
u max: 2.3276000918425552
u max: 2.3273996368803433
u max: 2.3271639129555752
u max: 2.326892932822208
u max: 2.326586720168286
u max: 2.326245309737395
u max: 2.325868747462069
u max: 2.3254570906059904
u max: 2.3250104079119
u max: 2.324528779752169
u max: 2.3240122982790252
u max: 2.3234610675711873
u max: 2.32287520377356
u max: 2.3222548352263868
u max: 2.3216001025802346
u max: 2.3209111588934244
u max: 2.320188169709224
u max: 2.319431313111077
u max: 2.3186407797557105
u max: 2.317816772885674
u max: 2.3169595083246257
u max: 2.3160692144605135
u max: 2.3151461322228615
u max: 2.3141905150613273
u max: 2.3132026289322223
u max: 2.312182752298837
u max: 2.3111311761491815
u max: 2.31004820403218
u max: 2.308934152110198
u max: 2.3077893492228236
u max: 2.306614136954377
u max: 2.3054088696962927
u max: 2.304173914695424
u max: 2.302909652080707
u max: 2.301616474862985
u max: 2.3002947889060303
u max: 2.

u max: 2.3281250745988085
u max: 2.328267104887789
u max: 2.3283742211535157
u max: 2.328446332123638
u max: 2.328483357243757
u max: 2.3284852266338856
u max: 2.3284518810516173
u max: 2.3283832718673194
u max: 2.328279361055597
u max: 2.328140121206268
u max: 2.3279655355566757
u max: 2.327755598046053
u max: 2.3275103133913144
u max: 2.327229697182707
u max: 2.3269137759968417
u max: 2.3265625875242693
u max: 2.3261761807084262
u max: 2.3257546158927784
u max: 2.3252979649729864
u max: 2.324806311550912
u max: 2.324279751087157
u max: 2.3237183910486547
u max: 2.323122351047618
u max: 2.322491762967985
u max: 2.321826771075672
u max: 2.321127532109341
u max: 2.320394215349293
u max: 2.3196270026634087
u max: 2.318826088530628
u max: 2.3179916800444147
u max: 2.3171239969003725
u max: 2.3162232713738082
u max: 2.3152897482941412
u max: 2.314323685023343
u max: 2.3133253514450947
u max: 2.312295029969859
u max: 2.311233015558687
u max: 2.310139615765661
u max: 2.309015150795721
u max:

u max: 2.326826476882112
u max: 2.327167644194144
u max: 2.3274745960354646
u max: 2.327747178821095
u max: 2.3279852497580538
u max: 2.328188676884893
u max: 2.3283573390830004
u max: 2.3284911260641197
u max: 2.328589938340071
u max: 2.328653687181424
u max: 2.3286822945720873
u max: 2.3286756931663386
u max: 2.328633826254188
u max: 2.3285566477401587
u max: 2.3284441221394796
u max: 2.328296224594504
u max: 2.328112940913012
u max: 2.32789426762862
u max: 2.327640212082516
u max: 2.3273507925246344
u max: 2.3270260382317036
u max: 2.326665989639226
u max: 2.3262706984841617
u max: 2.325840227955146
u max: 2.3253746528469543
u max: 2.324874059715988
u max: 2.3243385470332854
u max: 2.3237682253314
u max: 2.323163217341282
u max: 2.3225236581152715
u max: 2.3218496951324568
u max: 2.3211414883833585
u max: 2.320399210431864
u max: 2.319623046453841
u max: 2.318813194253497
u max: 2.317969864260585
u max: 2.3170932795131907
u max: 2.3161836756323853
u max: 2.3152413007959205
u max: 2.